In [1]:
from pathlib import Path
import shutil

# 기준 경로
BASE = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/250904_chunanCC")

# 상위 폴더 (이미지를 꺼내서 올려둘 위치)
PARENT = BASE.parent

# 이미지 확장자
IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp", ".JPG", ".JPEG", ".PNG", ".BMP"}

# 모든 하위 이미지 탐색
all_images = [p for p in BASE.rglob("*") if p.suffix in IMG_EXTS]

print(f"총 이미지 개수: {len(all_images)}")

# 이미지 옮기기
for img in all_images:
    dst = PARENT / img.name
    # 이름 충돌 시 뒤에 숫자 붙이기
    counter = 1
    while dst.exists():
        dst = PARENT / f"{img.stem}_{counter}{img.suffix}"
        counter += 1
    shutil.move(str(img), str(dst))

print("✅ 모든 이미지를 상위 폴더로 이동 완료")

# 이제 빈 폴더 삭제 (자식부터 먼저 지우기 위해 reverse sort)
for folder in sorted(BASE.rglob("*"), key=lambda p: len(str(p)), reverse=True):
    if folder.is_dir() and not any(folder.iterdir()):
        folder.rmdir()

# 마지막으로 BASE 폴더가 비었으면 삭제
if not any(BASE.iterdir()):
    BASE.rmdir()

print("✅ 하위 폴더 삭제 완료")


총 이미지 개수: 11160
✅ 모든 이미지를 상위 폴더로 이동 완료
✅ 하위 폴더 삭제 완료


In [4]:
from pathlib import Path
import os

# 경로 설정
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_data_use")

# 이미지 확장자
img_exts = {".jpg", ".jpeg", ".png", ".bmp"}

# 정렬된 이미지 리스트
images = sorted([p for p in SRC.rglob("*") if p.suffix.lower() in img_exts])

print(f"[INFO] Found {len(images)} images")

# === 이름 바꾸기 ===
prefix = "20250904"  # 원하는 접두사
start_idx = 1

for i, img in enumerate(images, start=start_idx):
    new_name = f"{prefix}_{i:05d}{img.suffix.lower()}"  # 5자리 번호
    new_path = img.parent / new_name
    os.rename(img, new_path)

print("[DONE] 파일 이름 변경 완료 ✅")

[INFO] Found 11160 images
[DONE] 파일 이름 변경 완료 ✅


In [1]:
# ============================================================
# YOLOv8 자동 라벨링 + 통계 출력
# Src : /home/dw/ws_job_msislab/Golf_Project/data/20250902_data_start_label
# Out : 같은 폴더에 동일 stem의 .txt 파일 (YOLO xywh normalized)
# Model : good_data_2_aug_train/weights/best.pt
# Classes : 0=Divot, 1=Fixed_Divot, 2=Dieased_grass
# ============================================================

from ultralytics import YOLO
from pathlib import Path

# 경로 설정
SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250904/20250904_data_use")
WEIGHT = "/home/dw/ws_job_msislab/Golf_Project/runs_yolo/good_data_2_aug_train/weights/best.pt"

# 모델 로드
model = YOLO(WEIGHT)

# 이미지 파일 모으기
img_exts = {".jpg", ".jpeg", ".png", ".bmp"}
images = [p for p in SRC.rglob("*") if p.suffix.lower() in img_exts]

print(f"[INFO] Found {len(images)} images")

# === 집계용 변수 ===
total_images = len(images)
labeled_images = 0
total_labels = 0

# 라벨 저장 함수 (박스가 없으면 파일 자체를 만들지 않음)
def save_yolo_labels(result, save_dir):
    global labeled_images, total_labels
    stem = Path(result.path).stem
    label_path = Path(save_dir) / f"{stem}.txt"

    lines = []
    for box in result.boxes:
        cls_id = int(box.cls[0].item())
        conf   = float(box.conf[0].item())
        cx, cy, w, h = box.xywhn[0].tolist()  # [cx, cy, w, h], normalized

        # 원하는 confidence threshold 적용
        if conf >= 0.75:
            lines.append(f"{cls_id} {cx:.6f} {cy:.6f} {w:.6f} {h:.6f}\n")

    if lines:  # 박스가 하나라도 있을 때만 파일 생성
        with open(label_path, "w") as f:
            f.writelines(lines)
        labeled_images += 1
        total_labels += len(lines)

# 추론 및 저장
for img in images:
    results = model(img, imgsz=896, conf=0.7, device=0, half=True, verbose=False)
    for r in results:
        save_yolo_labels(r, img.parent)  # 같은 폴더에 저장

# === 결과 출력 ===
print("====================================")
print(f"[RESULT] 총 이미지 수         : {total_images}")
print(f"[RESULT] 라벨링된 이미지 수   : {labeled_images}")
print(f"[RESULT] 생성된 라벨(박스) 수 : {total_labels}")
print("====================================")
print("[DONE] 자동 라벨링 완료 ✅")

[INFO] Found 11160 images
[RESULT] 총 이미지 수         : 11160
[RESULT] 라벨링된 이미지 수   : 4118
[RESULT] 생성된 라벨(박스) 수 : 5443
[DONE] 자동 라벨링 완료 ✅


In [2]:
# === 백그라운드 이미지 복사 스크립트 (SameFileError 방지) ===
from pathlib import Path
import shutil

SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/20250700/good_data_fix_label_processing")
DST = SRC / "background_images"
DST.mkdir(exist_ok=True)

IMG_EXTS = {".jpg", ".jpeg", ".png", ".bmp"}

count = 0
for img_path in SRC.rglob("*"):
    if img_path.suffix.lower() in IMG_EXTS:
        # 이미 DST 내부 파일이면 건너뜀
        if DST in img_path.parents:
            continue

        # 라벨 파일 경로
        label_path = img_path.with_suffix(".txt")

        # 라벨이 없거나 비어 있으면 백그라운드
        if not label_path.exists() or label_path.stat().st_size == 0:
            target_path = DST / img_path.name
            if not target_path.exists():  # 중복 방지
                shutil.copy2(img_path, target_path)
                count += 1

print(f"[RESULT] 총 {count}장의 백그라운드 이미지를 {DST}에 복사 완료.")

[RESULT] 총 153장의 백그라운드 이미지를 /home/dw/ws_job_msislab/Golf_Project/data/20250700/good_data_fix_label_processing/background_images에 복사 완료.
